# Setup

This notebook will work you through to setup [AKS (Azure Kubernetes Service)](https://azure.microsoft.com/en-us/services/kubernetes-service/) and [Kubeflow](https://www.kubeflow.org/).


## 0. Prerequisites

* Docker (if you create your own docker image) - To install, see [docker site](https://docs.docker.com/install/).
* Azure CLI - The easiest way is to use [Azure DSVM](https://azure.microsoft.com/en-us/services/virtual-machines/data-science-virtual-machines/) or [Azure cloud shell](https://azure.microsoft.com/en-us/features/cloud-shell/), where Azure CLI is installed by default.

## 1. Kubernetes Setup

#### 1.1 Create AKS cluster
First, make sure you signed in to use Azure CLI with a correct subscription.

In [ ]:
!az login

In [ ]:
!az account show

```
{
  "environmentName": "AzureCloud",
  "id": "...",
  "isDefault": true,
  "name": "YOUR-SUBSCRIPTION-NAME",  # Make sure if this is your subscription 
  "state": "Enabled",
  "tenantId": "...",
  "user": {
    "name": "...",
    "type": "user"
  }
}
```

If not, you can change the default subscription by running:

In [ ]:
!az account set --subscription <YOUR-SUBSCRIPTION-NAME-OR-ID>

Run following command to create a resource group and AKS cluster.
This example will create four `Standard_NC6` (GPU VM) nodes for the cluster.

In [ ]:
RG_NAME = "<YOUR-RESOURCE-GROUP-NAME>"
AKS_NAME = "<RESOURCE-NAME>"
LOCATION = "<RESOURCE-REGION>"

In [ ]:
# Create resource group
!az group create --name {RG_NAME} --location {LOCATION}

# Create AKS cluster
!az aks create \
    --resource-group {RG_NAME} \
    --name {AKS_NAME} \
    --node-count 4 \
    --node-vm-size Standard_NC6 \
    --enable-addons monitoring \
    --generate-ssh-keys

Note, creating AKS cluster may take few minutes. If the creation is successful, you'll see something like:
```
{
  "id": "/subscriptions/.../resourceGroups/<YOUR-RESOURCE-GROUP-NAME>",
  "location": <RESOURCE-REGION>,
  "managedBy": null,
  "name": "<YOUR-RESOURCE-GROUP-NAME>",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": null
}
{
  "aadProfile": null,
  "addonProfiles": {
    ...
    
}
```

If you get credential error, try
1. Create a new SP:

  `az ad sp create-for-rbac --skip-assignment`

2. Create AKS cluster by providing credentials created above:

  `az aks create ... --service-principal <AppID> --client-secret <Pwd>` 

#### 1.2 Setup Kubernetes CLI: kubectl

If you don't have `kubectl`, run following command to install it:

In [ ]:
!sudo az aks install-cli

To configure `kubectl`, get the credential by running:

In [ ]:
!az aks get-credentials --resource-group {RG_NAME} --name {AKS_NAME}

To verify connection, run:

In [ ]:
!kubectl get nodes 

If connection is success, you can see something like:

```
NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-17965807-0   Ready     agent     11m       v1.12.7
aks-nodepool1-17965807-1   Ready     agent     11m       v1.12.7
aks-nodepool1-17965807-2   Ready     agent     11m       v1.12.7
aks-nodepool1-17965807-3   Ready     agent     11m       v1.12.7
```

To change to different cluster, you may use:
```
# display list of contexts
kubectl config get-contexts
# display the current-context
kubectl config current-context
# change contexts
kubectl config use-context my-cluster-name 
```

#### 1.3 Setup Kubernetes Dashboard

Via [Kubernetes Dashboard UI](https://kubernetes.io/docs/tasks/access-application-cluster/web-ui-dashboard/),
you can deploy containerized applications as well as see workloads, services, storage, and logs.

1. To open the dashboard:
If your AKS cluster uses **RBAC**<sup>a</sup>, a `ClusterRoleBinding` must be created before you can correctly [access the dashboard](https://docs.microsoft.com/en-us/azure/aks/kubernetes-dashboard).

```
!kubectl create clusterrolebinding kubernetes-dashboard --clusterrole=cluster-admin --serviceaccount=kube-system:kubernetes-dashboard
```

2. To browse the dashboard:

```
az aks browse --resource-group ${RG_NAME} --name ${AKS_NAME}
```

The dashboard will be started from your browser. Or you can open it from http://127.0.0.1:8001/.
If you are using VM, make sure the port is being forwarded to your local machine.
 
<sub>
a. To check if your AKS cluster uses RBAC, goto https://resources.azure.com/
and see if "'enableRBAC': true" from your AKS.
</sub>

#### 1.4 Install NVIDIA device plugin
> For **AKS Engine**, NVIDIA device plugin will automatically installed with N-Series GPU clusters.

For **AKS**, install NVIDIA device plugin as follows:

In [ ]:
# For Kubernetes 1.10:
# kubectl apply -f https://raw.githubusercontent.com/nvidia/k8s-device-plugin/v1.10/nvidia-device-plugin.yml
    
# For Kubernetes 1.11 and above:
!kubectl apply -f https://raw.githubusercontent.com/nvidia/k8s-device-plugin/v1.11/nvidia-device-plugin.yml

## 2. Kubeflow Setup
**Kubeflow** is dedicated to making deployments of machine learning (ML) workflows
on Kubernetes simple, portable and scalable. More details can be found from https://www.kubeflow.org.

#### 2.1 Install ksonnet
Kubeflow makes use of ksonnet to help manage deployments. For more details, see [kubeflow doc](https://www.kubeflow.org/docs/components/ksonnet/).

First, download file by running:

In [ ]:
import os

os.environ["KS_VER"] = "0.13.1"
os.environ["KS_PKG"] = "ks_{}_linux_amd64".format(os.environ["KS_VER"])
os.environ["PATH"] = "{0}:{1}/bin/{2}".format(
    os.environ["PATH"],
    os.environ["HOME"],
    os.environ["KS_PKG"]
)

In [ ]:
%%bash

wget -O /tmp/${KS_PKG}.tar.gz https://github.com/ksonnet/ksonnet/releases/download/v${KS_VER}/${KS_PKG}.tar.gz
mkdir -p ${HOME}/bin
tar -xvf /tmp/$KS_PKG.tar.gz -C ${HOME}/bin

#### 2.2 Install Kubeflow

To download Kubeflow v0.4.1,

In [ ]:
os.environ['KUBEFLOW_SRC'] = os.path.realpath("kfsrc")  # path to download kubeflow source
os.environ['KUBEFLOW_APP'] = "kfapp"  # kubeflow app name

In [ ]:
%%bash

mkdir ${KUBEFLOW_SRC}
cd ${KUBEFLOW_SRC}

curl https://raw.githubusercontent.com/kubeflow/kubeflow/v0.4.1/scripts/download.sh | bash

Then, initialize a kubeflow app and deploy by running:

In [ ]:
%%bash 

${KUBEFLOW_SRC}/scripts/kfctl.sh init ${KUBEFLOW_APP} --platform none

cd ${KUBEFLOW_APP}
${KUBEFLOW_SRC}/scripts/kfctl.sh generate k8s
${KUBEFLOW_SRC}/scripts/kfctl.sh apply k8s

To confirm the deployment,

In [ ]:
!kubectl get pods -n kubeflow

To change the current namespace to `kubeflow`,

In [ ]:
!kubectl config set-context {AKS_NAME} --namespace=kubeflow

## 3. Storage Setup

A PersistentVolumeClaim (PVC) is a request for storage by a user. For details, see [persistent volumes with Azure files](https://docs.microsoft.com/en-us/azure/aks/azure-files-dynamic-pv).

Set Azurefile storage class and deploy

In [ ]:
!kubectl apply -f azure-file-sc.yaml
!kubectl apply -f azure-pvc-roles.yaml

In [ ]:
# To create PVC
!kubectl apply -f azure-file-pvc.yaml

In [ ]:
# To verify
!kubectl get pvc azurefile


## Further readings 
* [Azure Kubeflow labs github repo](https://github.com/Azure/kubeflow-labs)
